In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sb
import imblearn


df = pd.read_csv("../data/t00/data_t00.csv")
df_full = df.copy()
drop_cols = ["worker_id", "resp_worker_id", "prop_worker_id", "updated", "status", "job_id", "status", "timestamp", "rowid", "offer_dss", "offer", "offer_final"]
df = df[[col for col in df.columns if col not in drop_cols]]
df = df.dropna()

cols = [col for col in df.columns if col != "min_offer"] + ["min_offer"]

/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [14]:
# df_risk = pd.read_csv("../data/t00/export/result__risk.csv")
risk_aversion = 0 + (df["risk_q2"] > df["risk_q1"]) + (df["risk_q3"] > df["risk_q1"]) + (df["risk_q4"] > df["risk_q3"]) 
df["risk_aversion"] = risk_aversion

In [15]:
df.corr()["min_offer"]

cc_beta                          -0.047574
cc_criterion                      0.137012
cc_false_alarm_avg_click_delay   -0.010808
cc_false_alarm_rate               0.031175
cc_false_alarms                   0.031175
cc_hit_avg_click_delay           -0.142619
cc_hit_rate                      -0.181943
cc_hits                          -0.181943
cc_sensitivity                   -0.191991
cpc_expected_value               -0.080356
cpc_expected_value_max           -0.105406
cpc_expected_value_min           -0.042665
cpc_q1                           -0.058369
cpc_q10                           0.169796
cpc_q2                           -0.164347
cpc_q3                           -0.035873
cpc_q4                           -0.049450
cpc_q5                            0.052344
cpc_q6                            0.059273
cpc_q7                           -0.146307
cpc_q8                            0.042001
cpc_q9                           -0.013383
cpc_time_spent                   -0.090321
exp_ultimat

In [28]:
ras_cols = [f"ras_q{i+1}" for i in range(30)]
ras_low = (df[ras_cols] == -3).sum(axis=1)
ras_high = (df[ras_cols] == 3).sum(axis=1)
# df[["risk_aversion", "min_offer"]].corr()
df["ras_low"] = ras_low
df["ras_high"] = ras_high

df[["ras_low", "ras_high", "min_offer"]].corr()

,ras_low,ras_high,min_offer
ras_low,1.000000,0.507429,-0.032480
ras_high,0.507429,1.000000,-0.019264
min_offer,-0.032480,-0.019264,1.000000


In [33]:
cpc_cols = [f"cpc_q{i+1}" for i in range(10)]
cpc_left = (df[cpc_cols]==df[cpc_cols].min(axis=1)).sum(axis=1)
cpc_right = (df[cpc_cols]==df[cpc_cols].max(axis=1)).sum(axis=1)
df["cpc_left"] = cpc_left
df["cpc_right"] = cpc_right

df[["cpc_left", "cpc_right", "min_offer"]].corr()

,cpc_left,cpc_right,min_offer
cpc_left,NaN,NaN,NaN
cpc_right,NaN,NaN,NaN
min_offer,NaN,NaN,1.0


In [500]:
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from core.utils.preprocessing import df_to_xydf
from core.models.metrics import avg_loss_ratio

def my_scoring(model, X, y):
    yPred = model.predict(X)
    print(np.unique(yPred))
    return avg_loss_ratio(y, yPred)

df_features, df_target = df_to_xydf(df)
y = df_target.values.ravel()
clf = PCA(4)
pca = clf.fit_transform(df_features)
model = RandomForestClassifier(50)
model = MLPClassifier(activation="logistic", solver="sgd", max_iter=1000)
# model = LogisticRegression(solver="liblinear", multi_class="auto")
# model = KNeighborsClassifier(26)
# model.fit(pca, df_target.values.ravel())
# model.predict(pca) == df_target.values.ravel()
cross_val_score(model, pca, y, scoring=my_scoring, cv=2).mean()

/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


[50]
[50]


0.184206827877811

In [541]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

model = LocalOutlierFactor(novelty=True, leaf_size=10, algorithm="brute")
split = 60
mask = y[:split] < 50
X_all = df_features.values
X = X_all[:split][mask]
model.fit(X)
yPred = model.predict(X_all)
print(yPred*y)

[ 40  50  40  50  50  50  40  45  60 -35 100 -50  40  10   5  40 -50  40
   0  40  40  45  50  50 -10  50  50 -50  45  25  50  40  40  20  35  50
 -50  40  45  50 -50  50   5  45  50  50  50  50   5   5  50  40   5  50
 -40  50  50  50  30  50  35  30  50  50  45  40   0  20 -50 100  40  40
 -20  35  50   5  50   5 -40  25  50  50  50  40  50 100   5  55  45  35
  50  45  30 -50  50  15 -50   5  50  50  50  50  40]


/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


In [546]:
from core.models import ClusterExtModel, ClusterModel
from core.utils.selection import ffs, process_benchmark_cv

model = ClusterExtModel()

# ffs(model, df, early_stop=0)
process_benchmark_cv(model, X_all, y)

,avg_loss,avg_loss_ratio,avg_win_loss,invariance,loss_sum,mse,rejection_ratio
0,13.095238,0.241578,12.368421,1.0,275.0,417.857143,0.095238
1,7.380952,0.102680,7.380952,1.0,155.0,177.380952,0.000000
2,12.380952,0.148627,12.380952,1.0,260.0,445.238095,0.000000
3,14.250000,0.225952,15.000000,1.0,285.0,481.250000,0.050000
4,11.750000,0.219538,10.555556,1.0,235.0,408.750000,0.100000


In [557]:
from sklearn.preprocessing import LabelBinarizer, OrdinalEncoder, PolynomialFeatures, OneHotEncoder


x_ext = (OneHotEncoder(categories="auto").fit_transform(X_all)).todense()
x_ext

matrix([[0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 1., 1.],
        ...,
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 1., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 1., 1.]])

In [581]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
process_benchmark_cv(model, x_ext, y)

/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/franck/anaconda3/env

,avg_loss,avg_loss_ratio,avg_win_loss,invariance,loss_sum,mse,rejection_ratio
0,11.190476,0.218576,10.263158,0.335390,235,298.809524,0.095238
1,11.666667,0.185652,7.368421,0.348355,245,434.523810,0.095238
2,19.523810,0.291485,14.444444,0.348355,410,802.380952,0.142857
3,18.750000,0.319286,16.176471,0.239941,375,711.250000,0.150000
4,21.250000,0.411205,12.857143,0.238605,425,903.750000,0.300000


In [582]:
.10 * 200

20.0